# SQL Murder Mystery
This is a challenge I found on the internet that I have attempted.

It involves using SQL to follow the trail and find the murderer.

The link to the challenge is below

https://github.com/NUKnightLab/sql-mysteries

https://mystery.knightlab.com/#experienced

In [1]:
%load_ext sql

In [2]:
#Create connection to database file
%sql sqlite:///murder_mystery.db

In [3]:
%%sql
SELECT name, sql
  FROM sqlite_master
 WHERE type == 'table';

 * sqlite:///murder_mystery.db
Done.


name,sql
crime_scene_report,"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"
drivers_license,"CREATE TABLE drivers_license ( id integer PRIMARY KEY, age integer, height integer, eye_color text, hair_color text, gender text, plate_number text, car_make text, car_model text )"
person,"CREATE TABLE person ( id integer PRIMARY KEY, name text, license_id integer, address_number integer, address_street_name text, ssn integer, FOREIGN KEY (license_id) REFERENCES drivers_license(id) )"
facebook_event_checkin,"CREATE TABLE facebook_event_checkin ( person_id integer, event_id integer, event_name text, date integer, FOREIGN KEY (person_id) REFERENCES person(id) )"
interview,"CREATE TABLE interview ( person_id integer, transcript text, FOREIGN KEY (person_id) REFERENCES person(id) )"
get_fit_now_member,"CREATE TABLE get_fit_now_member ( id text PRIMARY KEY, person_id integer, name text, membership_start_date integer, membership_status text, FOREIGN KEY (person_id) REFERENCES person(id) )"
get_fit_now_check_in,"CREATE TABLE get_fit_now_check_in ( membership_id text, check_in_date integer, check_in_time integer, check_out_time integer, FOREIGN KEY (membership_id) REFERENCES get_fit_now_member(id) )"
income,"CREATE TABLE income ( ssn integer PRIMARY KEY, annual_income integer )"
solution,"CREATE TABLE solution ( user integer, value text )"


A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database.

In [4]:
%%sql
SELECT *
  FROM crime_scene_report
 WHERE (type == 'murder') & (city == 'SQL City') & (date == 20180115);

 * sqlite:///murder_mystery.db
Done.


date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


So this is the crime scene report we were after.
There are two witnesses at the crime scene.

First person:

In [5]:
%%sql
  SELECT *
    FROM person
   WHERE address_street_name == 'Northwestern Dr'
ORDER BY address_number desc
   LIMIT 1;

 * sqlite:///murder_mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


Second person:

In [6]:
%%sql
SELECT *
  FROM person
 WHERE (address_street_name == 'Franklin Ave') & (name LIKE '%Annabel%');

 * sqlite:///murder_mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


Now I want to have a look at their interviews and see what they said in their transcript

In [7]:
%%sql
SELECT p.id, p.name, i.transcript
  FROM person as p
  JOIN interview as i
    ON p.id = i.person_id
 WHERE p.id == 14887 OR p.id == 16371;

 * sqlite:///murder_mystery.db
Done.


id,name,transcript
14887,Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Key takeaway is:
* Membership number started with '48Z'
* Car number plate had 'H24W'
* Bag belongs to gold members
* Murderer goes to gym (last seen Jan 9th 2018 by Annabel)

So murderer `get_fit_now_check_in.check_in_date` = 20180109

Somehow related to `get_fit_now_member.id` = 48Z%

Somehow related to `drivers_licence.plate_number` = %H42W%

I want to start with who the car belongs to.

In [8]:
%%sql
SELECT p.id, p.name, d.plate_number
  FROM person as p
  JOIN drivers_license as d
    ON p.license_id = d.id
 WHERE d.plate_number LIKE '%H42W%';

 * sqlite:///murder_mystery.db
Done.


id,name,plate_number
51739,Tushar Chandra,4H42WR
67318,Jeremy Bowers,0H42W2
78193,Maxine Whitely,H42W0X


Now I want a list of members whos membership id starts with 48Z

In [9]:
%%sql
SELECT p.id, p.name, gfn.id as membership_id
  FROM person as p
  JOIN get_fit_now_member as gfn
    ON p.id = gfn.person_id
 WHERE gfn.id LIKE '48Z%';

 * sqlite:///murder_mystery.db
Done.


id,name,membership_id
49550,Tomas Baisley,48Z38
28819,Joe Germuska,48Z7A
67318,Jeremy Bowers,48Z55


I will have a look at who entered the gym on the date 9th Jan 2018

In [10]:
%%sql
SELECT p.id, p.name, gfn_check_in.*
  FROM person as p
  JOIN get_fit_now_member as gfn ON gfn.person_id == p.id
  JOIN get_fit_now_check_in as gfn_check_in ON gfn_check_in.membership_id == gfn.id
 WHERE (gfn_check_in.check_in_date == 20180109);

 * sqlite:///murder_mystery.db
Done.


id,name,membership_id,check_in_date,check_in_time,check_out_time
15247,Shondra Ledlow,X0643,20180109,957,1164
28073,Zackary Cabotage,UK1F2,20180109,344,518
55662,Sarita Bartosh,XTE42,20180109,486,1124
10815,Adriane Pelligra,1AE2H,20180109,461,944
83186,Burton Grippe,6LSTG,20180109,399,515
31523,Blossom Crescenzo,7MWHJ,20180109,273,885
92736,Carmen Dimick,GE5Q8,20180109,367,959
28819,Joe Germuska,48Z7A,20180109,1600,1730
67318,Jeremy Bowers,48Z55,20180109,1530,1700
16371,Annabel Miller,90081,20180109,1600,1700


In [11]:
%%sql
SELECT p.id, p.name, gfn_check_in.membership_id, gfn_check_in.check_in_date
  FROM person as p
  JOIN get_fit_now_member as gfn ON gfn.person_id == p.id
  JOIN get_fit_now_check_in as gfn_check_in ON gfn_check_in.membership_id == gfn.id
 WHERE (gfn_check_in.check_in_date == 20180109) 
   AND (p.name IN (SELECT c.name
                     FROM person as c
                     JOIN get_fit_now_member as gfn
                       ON c.id = gfn.person_id
                    WHERE gfn.id LIKE '48Z%'))
   AND (p.name IN (SELECT e.name
                     FROM person as e
                     JOIN drivers_license as d
                       ON e.license_id = d.id
                    WHERE d.plate_number LIKE '%H42W%'));

 * sqlite:///murder_mystery.db
Done.


id,name,membership_id,check_in_date
67318,Jeremy Bowers,48Z55,20180109


Jeremy Bowers is a common name amongst the three clues so all the evidence points towards Jeremy Bowers being the murder

In [12]:
%%sql
INSERT INTO solution VALUES (1, 'Jeremy Bowers');
     SELECT value FROM solution;

 * sqlite:///murder_mystery.db
1 rows affected.
Done.


value
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


In [13]:
%%sql
SELECT p.id, p.name, i.transcript
  FROM person as p
  JOIN interview as i
    ON p.id = i.person_id
 WHERE p.id == 67318;

 * sqlite:///murder_mystery.db
Done.


id,name,transcript
67318,Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


It seems Jeremy was a hitman. 

The actual murderer was:
* 5'5''/5'7'' (65''/67'')
* Red hair
* Drives a Tesla Model S
* Attended SQL Symphony Concert 3 times in December 2017

I am going to create a list of names for each criteria then use subqueries to see if there is any common names

In [14]:
%%sql 
SELECT p.id, p.name,d.gender, d.height, d.hair_color, d.car_make, d.car_model
  FROM person as p
  JOIN drivers_license as d
    ON p.license_id = d.id
 WHERE (d.height BETWEEN 65 AND 67)&(d.hair_color == 'red')&(d.car_make == 'Tesla')&(d.car_model == 'Model S');

 * sqlite:///murder_mystery.db
Done.


id,name,gender,height,hair_color,car_make,car_model
78881,Red Korb,female,65,red,Tesla,Model S
90700,Regina George,female,66,red,Tesla,Model S
99716,Miranda Priestly,female,66,red,Tesla,Model S


So we have 3 names here who match 3 out of 4 descriptions

In [15]:
%%sql
SELECT p.id, p.name, COUNT(e.event_name) as visits
  FROM person as p
  JOIN facebook_event_checkin as e
    ON p.id = e.person_id
 WHERE e.date LIKE '201712%'
   AND (e.event_name == 'SQL Symphony Concert')
GROUP BY p.id
ORDER BY visits desc
 LIMIT 5;

 * sqlite:///murder_mystery.db
Done.


id,name,visits
99716,Miranda Priestly,3
24556,Bryan Pardo,3
28582,Denyse Bloomgren,2
92343,Freida Betschart,1
81526,Odis Manteca,1


Miranda Priestly must be the murderer as she is fits all the descriptions given by the hitman

In [16]:
%%sql
INSERT INTO solution VALUES (1, 'Miranda Priestly');
     SELECT value FROM solution;

 * sqlite:///murder_mystery.db
1 rows affected.
Done.


value
"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"
